Recommendation Using Truncated SVD

In [2]:
import pandas as pd

In [33]:
df_svd=pd.read_csv('data_ML2.csv')

In [34]:
df_svd.shape

(200, 16)

In [35]:
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity

# Create a user-item matrix (User_ID x Dish_Name)
user_item_matrix = df_svd.pivot_table(index='User_ID', columns='Dish_Name', values='Order_Price').fillna(0)

In [36]:
# Apply SVD (Singular Value Decomposition)
svd = TruncatedSVD(n_components=26, random_state=42)
latent_matrix = svd.fit_transform(user_item_matrix)

# Calculate cosine similarities between dishes
dish_similarity = cosine_similarity(latent_matrix.T)
dish_similarity_df = pd.DataFrame(dish_similarity, index=user_item_matrix.columns, columns=user_item_matrix.columns)

In [39]:
def recommend_dishes(dish_name, user_preferences=None, season=None, excluded_ingredients=None, num_recommendations=5):
    # Start with the full dish similarity dataframe
    filtered_dishes = dish_similarity_df.copy()

    if user_preferences:
        preferred_dishes = df_svd[df_svd['User_Preferences'] == user_preferences]['Dish_Name'].unique()
        preferred_dishes = [dish for dish in preferred_dishes if dish in filtered_dishes.index]
        filtered_dishes = filtered_dishes.loc[preferred_dishes, preferred_dishes]

    if season:
        seasonal_dishes = df_svd[df_svd['Season'] == season]['Dish_Name'].unique()
        seasonal_dishes = [dish for dish in seasonal_dishes if dish in filtered_dishes.index]
        filtered_dishes = filtered_dishes.loc[seasonal_dishes, seasonal_dishes]

    if excluded_ingredients:
        excluded_ingredient_set = set(excluded_ingredients)
        ingredient_filtered_dishes = df_svd[~df_svd['Ingredients_List'].apply(lambda x: excluded_ingredient_set.intersection(set(x.split(', '))) != set())]['Dish_Name'].unique()
        ingredient_filtered_dishes = [dish for dish in ingredient_filtered_dishes if dish in filtered_dishes.index]
        filtered_dishes = filtered_dishes.loc[ingredient_filtered_dishes, ingredient_filtered_dishes]

    if dish_name not in filtered_dishes.index:
        return f"Dish '{dish_name}' not found in the filtered list."

    # Sorting dishes based on similarity and returning the top recommendations
    similar_dishes = filtered_dishes[dish_name].sort_values(ascending=False)[1:num_recommendations + 1]
    
    return similar_dishes

dish = input("Enter the dish name: ")
user_pref = input("Enter user preferences: ")
excluded_ingredients = input("Enter ingredients to exclude (comma-separated): ").split(',')

recommendations = recommend_dishes(dish, user_pref, excluded_ingredients=excluded_ingredients, num_recommendations=5)

if not recommendations.empty:
    dish_names = recommendations.index
    dishes = pd.DataFrame({"Recommended Dishes": dish_names})
else:
    dishes = pd.DataFrame({"Recommended Dishes": []})  # Create an empty DataFrame if no recommendations

print(dishes)


Enter the dish name:  Matar Paneer
Enter user preferences:  Vegetarian
Enter ingredients to exclude (comma-separated):  Potatoes


  Recommended Dishes
0     Sarson Ka Saag
1              Kadhi
2     Haldi Ki Sabzi
3       Palak Paneer
4              Pulao
